#### ConversationBufferMemory (Stores Everything)

In [1]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

load_dotenv()

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)

# Create memory that stores everything
memory = ConversationBufferMemory()

# Create conversation chain with memory
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True  # Shows what's being remembered
)

# Turn 1
print("="*60)
print("TURN 1:")
response1 = conversation.predict(input="Hi! My name is Jack and I love Python programming.")
print(f"AI: {response1}")

# Turn 2
print("\n" + "="*60)
print("TURN 2:")
response2 = conversation.predict(input="What's my name and what do I love?")
print(f"AI: {response2}")

# Turn 3
print("\n" + "="*60)
print("TURN 3:")
response3 = conversation.predict(input="Can you suggest a Python project for me?")
print(f"AI: {response3}")

# View memory contents
print("\n" + "="*60)
print("MEMORY CONTENTS:")
print(memory.load_memory_variables({}))


c:\Users\CHANDAN\Codes\Learning_codes\LangChain\Langchain_gemini\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TURN 1:


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi! My name is Jack and I love Python programming.
AI:

> Finished chain.
AI: Hello Jack! It's absolutely wonderful to meet you! Python programming, you say? That's fantastic! You've picked a truly incredible language.

As an AI, I spend a *lot* of my time processing and understanding information, and I can tell you, Python comes up *constantly* in the data I'm trained on. It's just so incredibly popular and versatile!

I've learned that one of the big reasons people, like yourself, love Python is its readability. That clean, intuitive syntax with its emphasis on whitespace makes it almost feel like you're writing in plain English, which is super helpfu

#### Method 2: ConversationBufferWindowMemory (Last N Messages)

In [ ]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

load_dotenv()

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)

# Store only last 2 interactions (4 messages: 2 user + 2 AI)
memory = ConversationBufferWindowMemory(k=2)

conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

# Multiple turns
print("Turn 1:")
conversation.predict(input="My name is Jack")

print("\nTurn 2:")
conversation.predict(input="I live in Bengaluru")

print("\nTurn 3:")
conversation.predict(input="I work as a developer")

print("\nTurn 4:")
response = conversation.predict(input="What's my name?")
print(f"\nAI Response: {response}")
# This might fail because Turn 1 is outside the window!

print("\n" + "="*60)
print("MEMORY CONTENTS (only last 2 interactions):")
print(memory.load_memory_variables({}))


#### Method 3: ConversationSummaryMemory (Summarizes History)

In [ ]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationChain

load_dotenv()

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)

# Memory that summarizes conversation
memory = ConversationSummaryMemory(llm=llm)

conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

# Have a long conversation
print("Turn 1:")
conversation.predict(input="Hi! I'm Jack, a Python developer from Bengaluru.")

print("\nTurn 2:")
conversation.predict(input="I'm learning LangChain to build AI applications.")

print("\nTurn 3:")
conversation.predict(input="My favorite frameworks are FastAPI and Django.")

print("\nTurn 4:")
conversation.predict(input="I also love working with databases like PostgreSQL.")

print("\nTurn 5:")
response = conversation.predict(input="Based on our conversation, what kind of projects would suit me?")
print(f"\nAI Response: {response}")

# View the summary
print("\n" + "="*60)
print("MEMORY SUMMARY:")
print(memory.load_memory_variables({}))


#### Method 4: ConversationSummaryBufferMemory (Best of Both Worlds)

In [ ]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationChain

load_dotenv()

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)

# Hybrid: summarize when exceeding 200 tokens
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=200
)

conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

# Long conversation
inputs = [
    "My name is Jack.",
    "I'm from Bengaluru, India.",
    "I work as a Python developer.",
    "I specialize in backend development with FastAPI.",
    "I'm currently learning LangChain and AI.",
    "What can you tell me about myself?"
]

for i, user_input in enumerate(inputs, 1): # enumerate(start=1)
    print(f"\n{'='*60}")
    print(f"TURN {i}:")
    response = conversation.predict(input=user_input)
    print(f"AI: {response}")

# View memory
print("\n" + "="*60)
print("MEMORY (Recent messages + Summary of old):")
print(memory.load_memory_variables({}))


#### Modern Way: Using LCEL with Memory (Manual Approach)

In [2]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage

load_dotenv()

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)

# Prompt with message history placeholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Remember the conversation context."),
    MessagesPlaceholder("chat_history"),  # History goes here
    ("human", "{input}")
])

# Create chain
chain = prompt | llm | StrOutputParser()

# Manually manage conversation history
chat_history = []

def chat(user_input: str) -> str:
    """Send a message and update history."""
    # Invoke chain with current history
    response = chain.invoke({
        "chat_history": chat_history,
        "input": user_input
    })

    # Update history
    chat_history.append(HumanMessage(content=user_input))
    chat_history.append(AIMessage(content=response))

    return response

# Test the conversation
print("Turn 1:")
print(f"AI: {chat('Hi! My name is Chandan.')}")

print("\nTurn 2:")
print(f"AI: {chat('I love Python programming.')}")

print("\nTurn 3:")
print(f"AI: {chat('What do you know about me?')}")

# View history
print("\n" + "="*60)
print("CHAT HISTORY:")
for msg in chat_history:
    role = "Human" if isinstance(msg, HumanMessage) else "AI"
    print(f"{role}: {msg.content}")


Turn 1:
AI: Hi Chandan! It's nice to meet you.

Turn 2:
AI: That's fantastic, Chandan! Python is a wonderful language.

What do you enjoy most about it, or what kind of Python projects do you work on?

Turn 3:
AI: Based on our conversation so far, Chandan, I know two things about you:

1.  Your name is **Chandan**.
2.  You **love Python programming**.

CHAT HISTORY:
Human: Hi! My name is Chandan.
AI: Hi Chandan! It's nice to meet you.
Human: I love Python programming.
AI: That's fantastic, Chandan! Python is a wonderful language.

What do you enjoy most about it, or what kind of Python projects do you work on?
Human: What do you know about me?
AI: Based on our conversation so far, Chandan, I know two things about you:

1.  Your name is **Chandan**.
2.  You **love Python programming**.


In [3]:
print(chat_history)

[HumanMessage(content='Hi! My name is Chandan.'), AIMessage(content="Hi Chandan! It's nice to meet you."), HumanMessage(content='I love Python programming.'), AIMessage(content="That's fantastic, Chandan! Python is a wonderful language.\n\nWhat do you enjoy most about it, or what kind of Python projects do you work on?"), HumanMessage(content='What do you know about me?'), AIMessage(content='Based on our conversation so far, Chandan, I know two things about you:\n\n1.  Your name is **Chandan**.\n2.  You **love Python programming**.')]


In [ ]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompt import  ChatPromptTemplate, MessagesPlaceholder
from langchain_core.outputparser import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage

load_dotenv()
llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.5)


chat_history=[]

prompt=ChatPromptTemplate.from_messages([
    ("sysytem","You are an helpful AI assistent, ansswer usng past conversation if needed"),
    MessagesPlaceholder("chat_history"),
    ("Human","{input}")
])

chain= prompt | llm | StrOutputParser()

def chat(user_input: str) ->str:

    response=chain.invoke({
        "chat_history": chat_history,
        "input":user_input
            })

    chat_history.append(HumanMessage(content=user_input))
    chat_history.append(AIMessage(content=response))

    return response

print("Turn_1")
print(f"AI: {chat('Hi my name is jack')}")

print("Turn_2")
print(f"Ai: {chat('I am from Bengaluru')}")

print("turn_5")
print(f"Ai: Tell me what do you know aboyt me ")

print("ChatIstory\n","="*30)
for msg in chat_history:
    role="Human" if isinstance(msg,HumanMessage) else "AI"
    print(f"{role}: {msg.content}")

#### Modern Way: RunnableWithMessageHistory

In [ ]:
from dotenv import load_dotenv

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

# 1. Load environment variables
load_dotenv()

# 2. Create LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.7
)

# 3. Create prompt with history placeholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Remember previous messages."),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

# 4. Create base (stateless) chain
chain = prompt | llm | StrOutputParser()

# 5. Session store (in-memory)
store = {}

def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# 6. Wrap chain with automatic message history
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

# 7. Choose a session
config = {"configurable": {"session_id": "abc123"}}

# 8. Multi-turn conversation
print("Turn 1:")
print("AI:", chain_with_history.invoke(
    {"input": "Hi! My name is Chandan."},
    config=config
))

print("\nTurn 2:")
print("AI:", chain_with_history.invoke(
    {"input": "I love Python programming."},
    config=config
))

print("\nTurn 3:")
print("AI:", chain_with_history.invoke(
    {"input": "What do you know about me?"},
    config=config
))

# 9. Inspect stored history
print("\n" + "=" * 60)
print("STORED CHAT HISTORY:")

history = get_session_history("abc123")
for msg in history.messages:
    role = msg.type.upper()
    print(f"{role}: {msg.content}")


In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_history import ChatMwssageHIstory
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.outputparser import StrOutputParser

load_dotenv()

chat_history=[]

llm=ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temparature=0
)

prompt=ChatPromptTemplate.from_messages([
    ("system","you are an helpful AI assistent with past conversation history"),
    MessagesPlaceholder("chat_history"),
    ("Human","{input}")
])

chain= prompt|llm|StrOutputParser()

session={}

def get_session_history(session_id :str):
    if session_id not in strore:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

chain_with_history=RunnableWithMessageHIstory(
    chain,
    get_session_history,
    input_message_key="input",
    history_message_key="chat_history"

)

config={"configurable":{"session_id":"abc123"}}

response=chain_with_history.invoke({"input": "Hello"}, config=config)

print(response)